# Day 6 - Lab 1: Building RAG Systems

**Objective:** Build a RAG (Retrieval-Augmented Generation) system orchestrated by LangGraph, scaling in complexity from a simple retriever to a multi-agent team that includes a grader and a router.

**Estimated Time:** 180 minutes

**Introduction:**
Welcome to Day 6! Today, we build one of the most powerful and common patterns for enterprise AI: a system that can answer questions about your private documents. We will use LangGraph to create a 'research team' of AI agents. Each agent will have a specific job, and LangGraph will act as the manager, orchestrating their collaboration to find the best possible answer.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We need several libraries for this lab. `langgraph` is the core orchestrator, `langchain` provides the building blocks, `faiss-cpu` is for our vector store, and `pypdf` is for loading documents.

**Model Selection:**
For RAG and agentic workflows, models with strong instruction-following and reasoning are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `load_artifact()`: To read the project documents that will form our knowledge base.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langgraph')
install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('faiss-cpu')
install_if_missing('pypdf')

from utils import setup_llm_client, load_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

faiss-cpu not found, installing...
pypdf not found, installing...


2025-11-04 13:23:08,817 ag_aisoftdev.utils WARNING Optional core dependencies not found. Some features will be degraded. provider=None model=None latency_ms=None artifacts_path=None
2025-11-04 13:23:08,817 ag_aisoftdev.utils WARNING To enable full functionality run: pip install python-dotenv ipython plantuml provider=None model=None latency_ms=None artifacts_path=None
2025-11-04 13:23:08,836 ag_aisoftdev.utils WARNING python-dotenv not installed; .env will not be loaded. provider=None model=None latency_ms=None artifacts_path=None
2025-11-04 13:23:09,243 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: Building the Knowledge Base

An agent is only as smart as the information it can access. We will create a vector store containing all the project artifacts we've created so far. This will be our agent's 'knowledge base'.

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def create_knowledge_base(file_paths):
    """Loads documents from given paths and creates a FAISS vector store.""" 
    all_docs = []
    for path in file_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata={"source": path} # Add source metadata
            all_docs.extend(docs)
        else:
            print(f"Warning: Artifact not found at {full_path}")

    if not all_docs:
        print("No documents found to create knowledge base.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    
    print(f"Creating vector store from {len(splits)} document splits...")
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

all_artifact_paths = ["artifacts/day1_prd.md", "artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
retriever = create_knowledge_base(all_artifact_paths)

Creating vector store from 33 document splits...


## Step 3: The Challenges

### Challenge 1 (Foundational): A Simple RAG Graph

**Task:** Build a simple LangGraph with two nodes: one to retrieve documents and one to generate an answer.

> **Tip:** Think of `AgentState` as the shared 'whiteboard' for your agent team. Every agent (or 'node' in the graph) can read from and write to this state, allowing them to pass information to each other as they work on a problem.

**Instructions:**
1.  Define the state for your graph using a `TypedDict`. It should contain keys for `question` and `documents`.
2.  Create a "Retriever" node. This is a Python function that takes the state, uses the `retriever` to get relevant documents, and updates the state with the results.
3.  Create a "Generator" node. This function takes the state, creates a prompt with the question and retrieved documents, calls the LLM, and stores the answer.
4.  Build the `StateGraph`, add the nodes, and define the edges (`RETRIEVE` -> `GENERATE`).
5.  Compile the graph and invoke it with a question about your project.

**Expected Quality:** A functional graph that can answer a simple question (e.g., "What is the purpose of this project?") by retrieving context from the project artifacts.

In [10]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_core.documents import Document

# Step 1: Define the state for the graph using TypedDict
class AgentState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

# Step 2: Create a "Retriever" node
def retrieve_node(state: AgentState) -> AgentState:
    """Retrieves relevant documents based on the question."""
    question = state["question"]
    print(f"🔍 Retrieving documents for: {question}")
    
    # Use the retriever to get relevant documents
    documents = retriever.invoke(question)
    
    print(f"📄 Found {len(documents)} relevant documents")
    
    # Update state with retrieved documents
    state["documents"] = documents
    return state

# Step 3: Create a "Generator" node
def generate_node(state: AgentState) -> AgentState:
    """Generates an answer using the question and retrieved documents."""
    question = state["question"]
    documents = state["documents"]
    
    print(f"🤖 Generating answer...")
    
    # Create context from retrieved documents
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # Create prompt with question and retrieved documents
    prompt = f"""Based on the following context, answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer:"""
    
    # Call the LLM to generate response
    if api_provider == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )
        answer = response.choices[0].message.content
    else:
        # Handle other providers if needed
        answer = "Error: Provider not supported in this example"
    
    print(f"✅ Answer generated")
    
    # Store the answer in state
    state["answer"] = answer
    return state

# Step 4: Build the StateGraph, add nodes, and define edges
def create_simple_rag_graph():
    """Creates a simple RAG graph with retrieve and generate nodes."""
    
    # Create the state graph
    workflow = StateGraph(AgentState)
    
    # Add nodes to the graph
    workflow.add_node("RETRIEVE", retrieve_node)
    workflow.add_node("GENERATE", generate_node)
    
    # Define the edges: RETRIEVE -> GENERATE
    workflow.set_entry_point("RETRIEVE")
    workflow.add_edge("RETRIEVE", "GENERATE")
    workflow.add_edge("GENERATE", END)
    
    # Compile the graph
    app = workflow.compile()
    
    return app

# Step 5: Compile the graph and invoke it with a question
print("🚀 Building Simple RAG Graph...")
rag_app = create_simple_rag_graph()

# Test with a question about the project
test_question = "What is the purpose of this project?"
print(f"\n📋 Testing with question: '{test_question}'")
print("="*50)

# Invoke the graph
result = rag_app.invoke({
    "question": test_question,
    "documents": [],
    "answer": ""
})

print("="*50)
print("🎯 Final Answer:")
# Use text wrapping for better readability
print_wrapped(result["answer"])

print("\n🔍 Retrieved Documents:")
for i, doc in enumerate(result["documents"], 1):
    print(f"{i}. Source: {doc.metadata.get('source', 'Unknown')}")
    print_wrapped(f"Content preview: {doc.page_content[:100]}...", "   ", width=77)

🚀 Building Simple RAG Graph...

📋 Testing with question: 'What is the purpose of this project?'
🔍 Retrieving documents for: What is the purpose of this project?
📄 Found 4 relevant documents
🤖 Generating answer...
📄 Found 4 relevant documents
🤖 Generating answer...
✅ Answer generated
🎯 Final Answer:
The purpose of this project is to revolutionize the company's onboarding process
by creating a centralized, intuitive platform that addresses the current
challenges of fragmented information and manual administrative tasks. The New
Hire Experience Platform aims to provide a seamless, engaging, and efficient
onboarding journey for new employees, enabling them to feel prepared, connected,
and productive from day one, while also reducing the administrative burden on HR
and hiring managers.

🔍 Retrieved Documents:
1. Source: artifacts/day1_prd.md
   Content preview: ## 2. The Problem A detailed look at the pain points this
   product will solve. This section justifies...
2. Source: artifacts/day

In [9]:
# Helper function for text wrapping to improve readability
import textwrap

def print_wrapped(text, prefix="", width=80):
    """Print text with wrapping to avoid horizontal scrolling."""
    if prefix:
        # For prefixed text, use subsequent_indent to align continuation lines
        wrapped = textwrap.fill(f"{prefix}{text}", width=width, 
                               subsequent_indent=" " * len(prefix))
    else:
        wrapped = textwrap.fill(text, width=width)
    print(wrapped)

### Challenge 2 (Intermediate): A Graph with a Grader Agent

**Task:** Add a second agent to your graph that acts as a "Grader," deciding if the retrieved documents are relevant enough to answer the question.

> **What is a conditional edge?** It's a decision point. After a node completes its task (like our 'Grader'), the conditional edge runs a function to decide which node to go to next. This allows your agent to change its plan based on new information.

**Instructions:**
1.  Keep your `RETRIEVE` and `GENERATE` nodes from the previous challenge.
2.  Create a new "Grader" node. This function takes the state (question and documents) and calls an LLM with a specific prompt: "Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'."
3.  Add a **conditional edge** to your graph. After the `RETRIEVE` node, the graph should go to the `GRADE` node. After the `GRADE` node, it should check the grader's response. If 'yes', it proceeds to the `GENERATE` node. If 'no', it goes to an `END` node, concluding that it cannot answer the question.

**Expected Quality:** A more robust graph that can gracefully handle cases where its knowledge base doesn't contain the answer, preventing it from hallucinating.

In [ ]:
# Challenge 2: RAG Graph with Grader Agent and Conditional Edges

# Step 1: Update the state to include grader results
class GradedAgentState(TypedDict):
    question: str
    documents: List[Document]
    answer: str
    grade: str  # New field for grader response

# Step 2: Keep the RETRIEVE and GENERATE nodes from Challenge 1 (with updated state)
def retrieve_node_v2(state: GradedAgentState) -> GradedAgentState:
    """Retrieves relevant documents based on the question."""
    question = state["question"]
    print(f"🔍 Retrieving documents for: {question}")
    
    # Use the retriever to get relevant documents
    documents = retriever.invoke(question)
    
    print(f"📄 Found {len(documents)} relevant documents")
    
    # Update state with retrieved documents
    state["documents"] = documents
    return state

def generate_node_v2(state: GradedAgentState) -> GradedAgentState:
    """Generates an answer using the question and retrieved documents."""
    question = state["question"]
    documents = state["documents"]
    
    print(f"🤖 Generating answer...")
    
    # Create context from retrieved documents
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # Create prompt with question and retrieved documents
    prompt = f"""Based on the following context, answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer:"""
    
    # Call the LLM to generate response
    if api_provider == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )
        answer = response.choices[0].message.content
    else:
        answer = "Error: Provider not supported in this example"
    
    print(f"✅ Answer generated")
    
    # Store the answer in state
    state["answer"] = answer
    return state

# Step 3: Create the new "Grader" node
def grade_node(state: GradedAgentState) -> GradedAgentState:
    """Grades if the retrieved documents are sufficient to answer the question."""
    question = state["question"]
    documents = state["documents"]
    
    print(f"⚖️  Grading document relevance...")
    
    # Create context from retrieved documents for grading
    context = "\n\n".join([f"Document {i+1}: {doc.page_content[:500]}" 
                          for i, doc in enumerate(documents)])
    
    # Improved grader prompt - more specific instructions
    grade_prompt = f"""You are a document relevance grader. Your task is to determine if the provided documents contain enough relevant information to answer the given question.

Question: {question}

Documents:
{context}

Instructions:
- Answer "yes" if the documents contain relevant information that can be used to answer the question
- Answer "no" if the documents do not contain relevant information to answer the question
- Be objective and focus on content relevance, not perfect completeness

Decision (yes/no):"""
    
    # Call the LLM to grade
    if api_provider == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": grade_prompt}],
            temperature=0.0  # Use 0 temperature for consistent grading
        )
        grade = response.choices[0].message.content.strip().lower()
        # Clean the response to ensure it's just "yes" or "no"
        if "yes" in grade:
            grade = "yes"
        else:
            grade = "no"
    else:
        grade = "no"  # Default to no if provider not supported
    
    print(f"📊 Grader decision: {grade}")
    
    # Store the grade in state
    state["grade"] = grade
    return state

# Step 4: Create conditional edge function
def decide_next_step(state: GradedAgentState) -> str:
    """Decides whether to generate answer or end based on grader decision."""
    grade = state.get("grade", "no")
    
    if grade == "yes":
        print("✅ Grade: YES - Proceeding to generate answer")
        return "GENERATE"
    else:
        print("❌ Grade: NO - Insufficient information, ending")
        return "END"

# Step 5: Build the enhanced StateGraph with conditional edges
def create_graded_rag_graph():
    """Creates a RAG graph with grader and conditional edges."""
    
    # Create the state graph
    workflow = StateGraph(GradedAgentState)
    
    # Add nodes to the graph
    workflow.add_node("RETRIEVE", retrieve_node_v2)
    workflow.add_node("GRADE", grade_node)
    workflow.add_node("GENERATE", generate_node_v2)
    
    # Define the flow: RETRIEVE -> GRADE -> (conditional) -> GENERATE or END
    workflow.set_entry_point("RETRIEVE")
    workflow.add_edge("RETRIEVE", "GRADE")
    
    # Add conditional edge after GRADE node
    workflow.add_conditional_edges(
        "GRADE",
        decide_next_step,
        {
            "GENERATE": "GENERATE",
            "END": END
        }
    )
    
    # GENERATE always goes to END
    workflow.add_edge("GENERATE", END)
    
    # Compile the graph
    app = workflow.compile()
    
    return app

# Step 6: Test the graded RAG system
print("\n🚀 Building Graded RAG Graph with Conditional Edges...")
graded_rag_app = create_graded_rag_graph()

print("\n" + "="*60)
print("🧪 TEST 1: Question that should have sufficient information")
print("="*60)

# Test with a question that should have good documents
test_question_1 = "What is the purpose of this project?"
print(f"📋 Testing with: '{test_question_1}'")

result_1 = graded_rag_app.invoke({
    "question": test_question_1,
    "documents": [],
    "answer": "",
    "grade": ""
})

print(f"\n🎯 Result:")
if result_1.get("answer"):
    print("Answer:")
    print_wrapped(result_1['answer'], width=80)  # Show full answer without truncation
else:
    print("No answer generated - insufficient information")

print("\n" + "="*60)
print("🧪 TEST 2: Question that should have insufficient information")
print("="*60)

# Test with a question that likely won't have good documents
test_question_2 = "What is the weather forecast for tomorrow in Tokyo?"
print(f"📋 Testing with: '{test_question_2}'")

result_2 = graded_rag_app.invoke({
    "question": test_question_2,
    "documents": [],
    "answer": "",
    "grade": ""
})

print(f"\n🎯 Result:")
if result_2.get("answer"):
    print("Answer:")
    print_wrapped(result_2['answer'], width=80)  # Show full answer without truncation
else:
    print("✅ Correctly identified insufficient information - no answer generated")

print("\n🎉 Challenge 2 Complete! The graded RAG system can now:")
print("   ✅ Evaluate document relevance before generating answers")
print("   ✅ Prevent hallucination by refusing to answer when information is insufficient")
print("   ✅ Use conditional edges to make intelligent routing decisions")


🚀 Building Graded RAG Graph with Conditional Edges...

🧪 TEST 1: Question that should have sufficient information
📋 Testing with: 'What is the purpose of this project?'
🔍 Retrieving documents for: What is the purpose of this project?
📄 Found 4 relevant documents
⚖️  Grading document relevance...
📊 Grader decision: yes
✅ Grade: YES - Proceeding to generate answer
🤖 Generating answer...
📊 Grader decision: yes
✅ Grade: YES - Proceeding to generate answer
🤖 Generating answer...
✅ Answer generated

🎯 Result:
Answer: The purpose of this project is to revolutionize the company's onboarding
process by creating a centralized, intuitive platform that addresses the current
challenges of fragmented information and manual administrative tasks. The New
Hire Experience Platform aims to provide a seamless, engaging, and e...

🧪 TEST 2: Question that should have insufficient information
📋 Testing with: 'What is the weather forecast for tomorrow in Tokyo?'
🔍 Retrieving documents for: What is the weathe

### Challenge 3 (Advanced): A Multi-Agent Research Team

**Task:** Build a sophisticated "research team" of specialized agents that includes a router to delegate tasks to the correct specialist.

**Instructions:**
1.  **Specialize your retriever:** Create two separate retrievers. One for the PRD (`prd_retriever`) and one for the technical documents (`tech_retriever` for schema and ADRs).
2.  **Define the Agents:**
    * `ProjectManagerAgent`: This will be the entry point and will act as a router. It uses an LLM to decide whether the user's question is about product requirements or technical details, and routes to the appropriate researcher.
    * `PRDResearcherAgent`: A node that uses the `prd_retriever`.
    * `TechResearcherAgent`: A node that uses the `tech_retriever`.
    * `SynthesizerAgent`: A node that takes the collected documents from either researcher and synthesizes a final answer.
3.  **Build the Graph:** Use conditional edges to orchestrate the flow: The entry point is the `ProjectManager`, which then routes to either the `PRD_RESEARCHER` or `TECH_RESEARCHER`. Both of those nodes should then route to the `SYNTHESIZE` node, which then goes to the `END`.

**Expected Quality:** A highly advanced agentic system that mimics a real-world research workflow, including a router and specialist roles, to improve the accuracy and efficiency of the RAG process.

In [ ]:
# Challenge 3: Multi-Agent Research Team with Specialized Retrievers and Router

# Step 1: Create specialized retrievers
def create_specialized_retrievers():
    """Creates specialized retrievers for different types of documents."""
    
    # PRD Retriever - Only for Product Requirements Document
    prd_paths = ["artifacts/day1_prd.md"]
    prd_docs = []
    for path in prd_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata = {"source": path, "type": "prd"}
            prd_docs.extend(docs)
    
    if prd_docs:
        prd_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        prd_splits = prd_splitter.split_documents(prd_docs)
        prd_vectorstore = FAISS.from_documents(documents=prd_splits, embedding=OpenAIEmbeddings())
        prd_retriever = prd_vectorstore.as_retriever()
        print(f"📋 PRD Retriever created with {len(prd_splits)} document splits")
    else:
        prd_retriever = None
        print("❌ No PRD documents found")
    
    # Tech Retriever - For schema and ADRs
    tech_paths = ["artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
    tech_docs = []
    for path in tech_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata = {"source": path, "type": "technical"}
            tech_docs.extend(docs)
    
    if tech_docs:
        tech_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        tech_splits = tech_splitter.split_documents(tech_docs)
        tech_vectorstore = FAISS.from_documents(documents=tech_splits, embedding=OpenAIEmbeddings())
        tech_retriever = tech_vectorstore.as_retriever()
        print(f"🔧 Tech Retriever created with {len(tech_splits)} document splits")
    else:
        tech_retriever = None
        print("❌ No technical documents found")
    
    return prd_retriever, tech_retriever

# Create specialized retrievers
prd_retriever, tech_retriever = create_specialized_retrievers()

# Step 2: Define the Multi-Agent State
class MultiAgentState(TypedDict):
    question: str
    route_decision: str  # "prd" or "technical"
    documents: List[Document]
    answer: str
    research_type: str  # Track which researcher was used

# Step 3: Define the Agent Nodes

def project_manager_agent(state: MultiAgentState) -> MultiAgentState:
    """Router agent that decides which specialist to use based on the question."""
    question = state["question"]
    print(f"👔 Project Manager analyzing: {question}")
    
    # Router prompt to classify the question
    router_prompt = f"""You are a project manager routing questions to the appropriate specialist.

Question: {question}

Analyze this question and determine if it's about:
- "prd": Product requirements, features, goals, user stories, or business objectives
- "technical": Technical implementation, database schema, architecture decisions, or technical specifications

Respond with exactly one word: either "prd" or "technical"

Classification:"""
    
    # Get routing decision from LLM
    if api_provider == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": router_prompt}],
            temperature=0.0
        )
        route_decision = response.choices[0].message.content.strip().lower()
        
        # Clean the response to ensure it's either "prd" or "technical"
        if "prd" in route_decision:
            route_decision = "prd"
        else:
            route_decision = "technical"
    else:
        route_decision = "prd"  # Default fallback
    
    print(f"🎯 Routing decision: {route_decision.upper()}")
    
    state["route_decision"] = route_decision
    return state

def prd_researcher_agent(state: MultiAgentState) -> MultiAgentState:
    """Specialized agent for researching product requirements."""
    question = state["question"]
    print(f"📋 PRD Researcher investigating: {question}")
    
    if prd_retriever:
        # Use PRD-specific retriever
        documents = prd_retriever.invoke(question)
        print(f"📄 Found {len(documents)} PRD-related documents")
        
        state["documents"] = documents
        state["research_type"] = "Product Requirements"
    else:
        print("❌ PRD retriever not available")
        state["documents"] = []
        state["research_type"] = "Product Requirements (unavailable)"
    
    return state

def tech_researcher_agent(state: MultiAgentState) -> MultiAgentState:
    """Specialized agent for researching technical details."""
    question = state["question"]
    print(f"🔧 Tech Researcher investigating: {question}")
    
    if tech_retriever:
        # Use technical-specific retriever
        documents = tech_retriever.invoke(question)
        print(f"📄 Found {len(documents)} technical documents")
        
        state["documents"] = documents
        state["research_type"] = "Technical Implementation"
    else:
        print("❌ Tech retriever not available")
        state["documents"] = []
        state["research_type"] = "Technical Implementation (unavailable)"
    
    return state

def synthesizer_agent(state: MultiAgentState) -> MultiAgentState:
    """Synthesizes the final answer from the collected research."""
    question = state["question"]
    documents = state["documents"]
    research_type = state.get("research_type", "Unknown")
    
    print(f"🧠 Synthesizer creating final answer from {research_type} research...")
    
    if not documents:
        state["answer"] = f"I apologize, but I couldn't find sufficient {research_type.lower()} information to answer your question."
        return state
    
    # Create context from research documents
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # Synthesis prompt that acknowledges the research type
    synthesis_prompt = f"""As a specialist synthesizer, create a comprehensive answer using the {research_type.lower()} research provided below.

Research Type: {research_type}
Question: {question}

Research Findings:
{context}

Instructions:
- Provide a clear, accurate answer based on the research
- Acknowledge the type of research conducted
- Be specific and cite relevant details from the findings
- If information is incomplete, state what additional research might be needed

Synthesized Answer:"""
    
    # Generate the final synthesized answer
    if api_provider == "openai":
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": synthesis_prompt}],
            temperature=0.1
        )
        answer = response.choices[0].message.content
    else:
        answer = "Error: Provider not supported in this example"
    
    print(f"✅ Synthesis complete")
    
    state["answer"] = answer
    return state

# Step 4: Create routing function for conditional edges
def route_to_researcher(state: MultiAgentState) -> str:
    """Routes to the appropriate researcher based on the manager's decision."""
    route_decision = state.get("route_decision", "prd")
    
    if route_decision == "prd":
        print("➡️  Routing to PRD Researcher")
        return "PRD_RESEARCHER"
    else:
        print("➡️  Routing to Tech Researcher")
        return "TECH_RESEARCHER"

# Step 5: Build the Multi-Agent Research Team Graph
def create_multi_agent_research_team():
    """Creates the sophisticated multi-agent research team graph."""
    
    # Create the state graph
    workflow = StateGraph(MultiAgentState)
    
    # Add all the agent nodes
    workflow.add_node("PROJECT_MANAGER", project_manager_agent)
    workflow.add_node("PRD_RESEARCHER", prd_researcher_agent)
    workflow.add_node("TECH_RESEARCHER", tech_researcher_agent)
    workflow.add_node("SYNTHESIZER", synthesizer_agent)
    
    # Define the flow
    # Entry point: PROJECT_MANAGER
    workflow.set_entry_point("PROJECT_MANAGER")
    
    # Conditional routing from PROJECT_MANAGER to researchers
    workflow.add_conditional_edges(
        "PROJECT_MANAGER",
        route_to_researcher,
        {
            "PRD_RESEARCHER": "PRD_RESEARCHER",
            "TECH_RESEARCHER": "TECH_RESEARCHER"
        }
    )
    
    # Both researchers route to SYNTHESIZER
    workflow.add_edge("PRD_RESEARCHER", "SYNTHESIZER")
    workflow.add_edge("TECH_RESEARCHER", "SYNTHESIZER")
    
    # SYNTHESIZER routes to END
    workflow.add_edge("SYNTHESIZER", END)
    
    # Compile the graph
    app = workflow.compile()
    
    return app

# Step 6: Test the Multi-Agent Research Team
print("\n🚀 Building Multi-Agent Research Team...")
research_team_app = create_multi_agent_research_team()

print("\n" + "="*70)
print("🧪 TEST 1: Product Requirements Question")
print("="*70)

prd_question = "What are the main goals and success metrics for this project?"
print_wrapped(f"📋 Testing PRD routing with: '{prd_question}'", width=70)

prd_result = research_team_app.invoke({
    "question": prd_question,
    "route_decision": "",
    "documents": [],
    "answer": "",
    "research_type": ""
})

print(f"\n🎯 PRD Research Result:")
print(f"Research Type: {prd_result.get('research_type', 'Unknown')}")
print("Answer:")
print_wrapped(prd_result['answer'], width=80)  # Show full answer without truncation

print("\n" + "="*70)
print("🧪 TEST 2: Technical Implementation Question")
print("="*70)

tech_question = "What database technology was chosen and why?"
print_wrapped(f"📋 Testing Tech routing with: '{tech_question}'", width=70)

tech_result = research_team_app.invoke({
    "question": tech_question,
    "route_decision": "",
    "documents": [],
    "answer": "",
    "research_type": ""
})

print(f"\n🎯 Tech Research Result:")
print(f"Research Type: {tech_result.get('research_type', 'Unknown')}")
print("Answer:")
print_wrapped(tech_result['answer'], width=80)  # Show full answer without truncation

print("\n" + "="*70)
print("🧪 TEST 3: Ambiguous Question (Router Decision)")
print("="*70)

ambiguous_question = "How will users interact with the system?"
print_wrapped(f"📋 Testing Router intelligence with: '{ambiguous_question}'", width=70)

ambiguous_result = research_team_app.invoke({
    "question": ambiguous_question,
    "route_decision": "",
    "documents": [],
    "answer": "",
    "research_type": ""
})

print(f"\n🎯 Router Decision Result:")
print(f"Routed to: {ambiguous_result.get('research_type', 'Unknown')}")
print("Answer:")
print_wrapped(ambiguous_result['answer'], width=80)  # Show full answer without truncation

print(f"\n🎉 Challenge 3 Complete! The Multi-Agent Research Team features:")
print("   ✅ Intelligent routing based on question content")
print("   ✅ Specialized retrievers for different document types")
print("   ✅ Collaborative workflow with distinct agent roles")
print("   ✅ Synthesis of research findings into comprehensive answers")
print("   ✅ Production-ready multi-agent architecture")

print(f"\n🏆 LAB COMPLETE! You've built a sophisticated RAG system that:")
print("   🔹 Started with simple retrieval and generation")
print("   🔹 Added quality control with grading")
print("   🔹 Evolved into a multi-agent research team")
print("   🔹 Demonstrates real-world enterprise AI patterns")

📋 PRD Retriever created with 20 document splits
🔧 Tech Retriever created with 13 document splits

🚀 Building Multi-Agent Research Team...

🧪 TEST 1: Product Requirements Question
📋 Testing PRD routing with: 'What are the main goals and success
metrics for this project?'
👔 Project Manager analyzing: What are the main goals and success metrics for this project?
🔧 Tech Retriever created with 13 document splits

🚀 Building Multi-Agent Research Team...

🧪 TEST 1: Product Requirements Question
📋 Testing PRD routing with: 'What are the main goals and success
metrics for this project?'
👔 Project Manager analyzing: What are the main goals and success metrics for this project?
🎯 Routing decision: PRD
➡️  Routing to PRD Researcher
📋 PRD Researcher investigating: What are the main goals and success metrics for this project?
📄 Found 4 PRD-related documents
🧠 Synthesizer creating final answer from Product Requirements research...
🎯 Routing decision: PRD
➡️  Routing to PRD Researcher
📋 PRD Researcher

## Lab Conclusion

Incredible work! You have now built a truly sophisticated AI system. You've learned how to create a knowledge base for an agent and how to use LangGraph to orchestrate a team of specialized agents to solve a complex problem. You progressed from a simple RAG chain to a system that includes quality checks (the Grader) and intelligent task delegation (the Router). These are the core patterns for building production-ready RAG applications.

> **Key Takeaway:** LangGraph allows you to define complex, stateful, multi-agent workflows as a graph. Using nodes for agents and conditional edges for decision-making enables the creation of sophisticated systems that can reason, delegate, and collaborate to solve problems more effectively than a single agent could alone.